<a href="https://colab.research.google.com/github/NguyenThanhDat10012004/define_image_in_pdf/blob/main/define_image_in_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!apt-get install -y poppler-utils
!pip install pdf2image
!pip install pillow
!pip install opencv-python


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.4 [186 kB]
Fetched 186 kB in 0s (484 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123576 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.4_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.4) ...
Setting up poppler-utils (22.02.0-2ubuntu0.4) ...
Processing triggers for man-db (2.10.2-1) ...


In [27]:
import cv2
import numpy as np
from pdf2image import convert_from_path
from PIL import Image
import plotly.graph_objects as go
from concurrent.futures import ThreadPoolExecutor

# Đường dẫn đến file PDF và hình ảnh cần tìm
pdf_path = '/content/Ngan hang cau hoi.pdf'  # Thay thế bằng đường dẫn thực tế
image_path = '/content/450240745_1668955677201051_6240235972198132789_n.png'  # Thay thế bằng đường dẫn thực tế

# Chuyển đổi các trang PDF thành hình ảnh
pages = convert_from_path(pdf_path, 100)  # Giảm độ phân giải xuống 100 DPI

# Đọc hình ảnh cần tìm
query_image = Image.open(image_path)
query_image = query_image.resize((query_image.width // 2, query_image.height // 2))  # Giảm độ phân giải hình ảnh cần tìm

# Hàm trích xuất đặc trưng từ hình ảnh sử dụng ORB và SIFT
def extract_features(image):
    gray = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)
    # ORB
    orb = cv2.ORB_create()
    orb_keypoints, orb_descriptors = orb.detectAndCompute(gray, None)
    # SIFT
    sift = cv2.SIFT_create()
    sift_keypoints, sift_descriptors = sift.detectAndCompute(gray, None)
    return (orb_keypoints, orb_descriptors), (sift_keypoints, sift_descriptors)

# Trích xuất đặc trưng từ hình ảnh cần tìm
(orb_query_keypoints, orb_query_descriptors), (sift_query_keypoints, sift_query_descriptors) = extract_features(np.array(query_image))

# Hàm so sánh đặc trưng và tính toán độ khớp sử dụng ORB và SIFT
def match_features(page_index, page_image):
    (orb_page_keypoints, orb_page_descriptors), (sift_page_keypoints, sift_page_descriptors) = extract_features(np.array(page_image))

    bf_orb = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches_orb = bf_orb.match(orb_query_descriptors, orb_page_descriptors)
    matches_orb = sorted(matches_orb, key=lambda x: x.distance)

    bf_sift = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matches_sift = bf_sift.match(sift_query_descriptors, sift_page_descriptors)
    matches_sift = sorted(matches_sift, key=lambda x: x.distance)

    # Kết hợp độ khớp của ORB và SIFT
    score = len(matches_orb) + len(matches_sift)

    # Sử dụng RANSAC để xác định tính đúng đắn của các điểm khớp
    if len(matches_orb) > 10 and len(matches_sift) > 10:
        src_pts_orb = np.float32([orb_query_keypoints[m.queryIdx].pt for m in matches_orb]).reshape(-1, 1, 2)
        dst_pts_orb = np.float32([orb_page_keypoints[m.trainIdx].pt for m in matches_orb]).reshape(-1, 1, 2)
        M_orb, mask_orb = cv2.findHomography(src_pts_orb, dst_pts_orb, cv2.RANSAC, 5.0)

        src_pts_sift = np.float32([sift_query_keypoints[m.queryIdx].pt for m in matches_sift]).reshape(-1, 1, 2)
        dst_pts_sift = np.float32([sift_page_keypoints[m.trainIdx].pt for m in matches_sift]).reshape(-1, 1, 2)
        M_sift, mask_sift = cv2.findHomography(src_pts_sift, dst_pts_sift, cv2.RANSAC, 5.0)

        if M_orb is not None and M_sift is not None:
            score += np.sum(mask_orb) + np.sum(mask_sift)

    return page_index, score

# Sử dụng xử lý song song để tăng tốc độ tìm kiếm
with ThreadPoolExecutor() as executor:
    results = list(executor.map(lambda i: match_features(i, pages[i]), range(len(pages))))

# Tạo danh sách độ khớp
matching_scores = [(page_index + 1, score) for page_index, score in results]

# Tạo biểu đồ tương tác với Plotly
pages, scores = zip(*matching_scores)
fig = go.Figure(data=go.Scatter(x=pages, y=scores, mode='markers+lines', text=[f'Page {p}, Score: {s}' for p, s in zip(pages, scores)], hoverinfo='text'))
fig.update_layout(title='Matching Score for Each Page in PDF',
                   xaxis_title='Page Number',
                   yaxis_title='Matching Score')
fig.show()
